In [16]:
print("check")

check


In [1]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 937.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def save_checkpoint(model, optimizer, epoch, batch_idx, path="checkpoint.pt"):
    torch.save({
        'epoch': epoch,
        'batch_idx': batch_idx,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, path)


In [4]:
def load_checkpoint(model, optimizer, path="checkpoint.pt"):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return checkpoint['epoch'], checkpoint['batch_idx']


In [5]:
from torch.utils.data import DataLoader, Dataset
import torch

data = torch.load(
    "/content/drive/MyDrive/sentiment_analysis/dataset/tokenized_with_labels.pt",
    weights_only=False
)




In [9]:
# build a map from original → contiguous
# label_map = {lab: i for i, lab in enumerate(sorted(unique_labels.tolist()))}
label_map = {0: 0, 1: 1, 3: 2}

# Apply it to your dataset
for item in data:
    item["labels"] = torch.tensor(label_map[item["labels"].item()], dtype=torch.long)

# Now the max label is 2, so
output_dim = len(label_map)  # = 3


In [10]:

# 2. Basic checks
print("Length of dataset:", len(data))
print("Type of first item:", type(data[0]))

# 3. Print the first item (sample)
print("First item:", data[0])

Length of dataset: 31726
Type of first item: <class 'dict'>
First item: {'input_ids': tensor([     2,   4384,   4285,   9545,     12,   3496,   1134, 109832,    477,
         26697, 134087,   7425,   2700,   4384,     31,   8351,     37,   2863,
          6511,  38637,   1883,   4285,   1640,   3496,   1134, 109832,     15,
         22186,   6169,    494,  22186,      3,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,   

In [11]:
from torch.utils.data import Dataset, DataLoader
import torch

# Load tokenized data
# tokenized_dataset = torch.load("/content/drive/MyDrive/sentiment_analysis/dataset/tokenized_by_indicbert_data_combined.pt")
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data  # list of dicts

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            'input_ids': item['input_ids'],
            'attention_mask': item['attention_mask'],
            'label': item['labels']
        }


dataset = MyDataset(data)
# dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


In [12]:
from torch.utils.data import random_split

# 80% training, 20% validation
train_size = int(0.8 * len(data))
val_size = len(data) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


In [13]:
train_dataset

In [14]:
print(type(train_dataset[0]))
print(dir(train_dataset[0]))  # See what's inside

<class 'dict'>
['__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__ior__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__or__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__ror__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'clear', 'copy', 'fromkeys', 'get', 'items', 'keys', 'pop', 'popitem', 'setdefault', 'update', 'values']


In [15]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

In [16]:
# train_features, train_labels = next(iter(dataloader))

In [17]:
def save_checkpoint(model, optimizer, epoch, batch_idx, path="checkpoint.pt"):
    torch.save({
        'epoch': epoch,
        'batch_idx': batch_idx,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, path)


In [18]:
def load_checkpoint(model, optimizer, path="checkpoint.pt"):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return checkpoint['epoch'], checkpoint['batch_idx']


In [19]:
import torch
import torch.nn as nn

class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim,
                 n_layers=1, bidirectional=False, dropout=0.3, pad_idx=0):
        super(SentimentLSTM, self).__init__()

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)

        # LSTM layer
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_dim,
            num_layers=n_layers,
            bidirectional=bidirectional,
            batch_first=True,
            dropout=dropout if n_layers > 1 else 0
        )

        # Fully connected output
        lstm_output_dim = hidden_dim * (2 if bidirectional else 1)
        self.fc = nn.Linear(lstm_output_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids, attention_mask=None):
        # input_ids: (batch_size, seq_len)
        embedded = self.dropout(self.embedding(input_ids))  # (batch_size, seq_len, embedding_dim)

        # Optionally apply attention mask: lengths from mask
        if attention_mask is not None:
            lengths = attention_mask.sum(dim=1).cpu()
            packed = nn.utils.rnn.pack_padded_sequence(embedded, lengths, batch_first=True, enforce_sorted=False)
            packed_output, (hidden, _) = self.lstm(packed)
        else:
            output, (hidden, _) = self.lstm(embedded)

        # If bidirectional, concatenate last forward and backward hidden states
        if self.lstm.bidirectional:
            hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)
        else:
            hidden = hidden[-1]

        out = self.fc(self.dropout(hidden))  # (batch_size, output_dim)
        return out


In [35]:
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    device = next(model.parameters()).device  # Get model's current device

    with torch.no_grad():
        for batch in dataloader:
            inputs = batch['input_ids'].to(device)
            labels = batch['label'].to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # Get predictions
            _, predicted = torch.max(outputs, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total if total > 0 else 0.0

    print(f"Validation Loss: {avg_loss:.4f}, Accuracy: {accuracy*100:.2f}%")

    model.train()
    return avg_loss


In [36]:
class EarlyStopping:
    def __init__(self, patience=3, delta=0.0):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True


In [37]:
vocab_size = 200000              # For BERT/IndicBERT vocab (if using tokenizer vocab)
embedding_dim = 128             # Smaller if not using pretrained embeddings
hidden_dim = 256
output_dim = 3                  # e.g., for 4 sentiment classes
n_layers = 10
bidirectional = True
dropout = 0.3
pad_idx = 0                     # Usually the token ID of [PAD]
num_epochs=20

model = SentimentLSTM(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    n_layers=n_layers,
    bidirectional=bidirectional,
    dropout=dropout,
    pad_idx=pad_idx
)


In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


SentimentLSTM(
  (embedding): Embedding(200000, 128, padding_idx=0)
  (lstm): LSTM(128, 256, num_layers=10, batch_first=True, dropout=0.3, bidirectional=True)
  (fc): Linear(in_features=512, out_features=3, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [39]:
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)


In [42]:
import os

checkpoint_dir = "/content/drive/MyDrive/sentiment_analysis/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = os.path.join(checkpoint_dir, "lstm_checkpoint_2.pt")


In [43]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define your LSTM model here
# model = SentimentLSTM()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

start_epoch, start_batch = 0, 0
# checkpoint_path = "checkpoint.pt"

# Try to load previous state
try:
    start_epoch, start_batch = load_checkpoint(model, optimizer, checkpoint_path)
    print(f"Resuming from epoch {start_epoch}, batch {start_batch}")
except FileNotFoundError:
    print("Starting from scratch.")

early_stopping = EarlyStopping(patience=3)

for epoch in range(start_epoch, num_epochs):
    model.train()

    for batch_idx, batch in enumerate(train_loader):
        if epoch == start_epoch and batch_idx < start_batch:
            continue

        # inputs = batch['input_ids']
        # labels = batch['label']

        inputs = batch['input_ids'].to(device)
        labels = batch['label'].to(device)


        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            save_checkpoint(model, optimizer, epoch, batch_idx, checkpoint_path)
            print(f"Checkpoint saved at epoch {epoch}, batch {batch_idx}")

    # Run validation at end of epoch
    val_loss = evaluate(model, val_loader, criterion)
    print(f"Epoch {epoch} Validation Loss: {val_loss:.4f}")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("🛑 Early stopping triggered.")
        break


Starting from scratch.
Checkpoint saved at epoch 0, batch 0
Checkpoint saved at epoch 0, batch 100
Checkpoint saved at epoch 0, batch 200
Checkpoint saved at epoch 0, batch 300
Validation Loss: 0.8761, Accuracy: 64.37%
Epoch 0 Validation Loss: 0.8761
Checkpoint saved at epoch 1, batch 0
Checkpoint saved at epoch 1, batch 100
Checkpoint saved at epoch 1, batch 200
Checkpoint saved at epoch 1, batch 300
Validation Loss: 0.8464, Accuracy: 66.50%
Epoch 1 Validation Loss: 0.8464
Checkpoint saved at epoch 2, batch 0
Checkpoint saved at epoch 2, batch 100
Checkpoint saved at epoch 2, batch 200
Checkpoint saved at epoch 2, batch 300
Validation Loss: 0.7872, Accuracy: 68.04%
Epoch 2 Validation Loss: 0.7872
Checkpoint saved at epoch 3, batch 0
Checkpoint saved at epoch 3, batch 100
Checkpoint saved at epoch 3, batch 200
Checkpoint saved at epoch 3, batch 300
Validation Loss: 0.7664, Accuracy: 69.43%
Epoch 3 Validation Loss: 0.7664
Checkpoint saved at epoch 4, batch 0
Checkpoint saved at epoch 4,

In [ ]:
from sklearn.metrics import classification_report

def evaluate_metrics(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            inputs = batch['input_ids'].to(device)
            labels = batch['label'].to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())

    print(classification_report(all_labels, all_preds, target_names=["Negative", "Neutral", "Positive"]))


In [ ]:
model.load_state_dict(torch.load("best_model.pt"))
test_loss = evaluate(model, test_loader, criterion)
print(f"Test Loss: {test_loss:.4f}")


In [46]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer

# Load tokenizer (adjust based on the one you used, e.g. "ai4bharat/indic-bert")
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your trained model definition
# (Make sure the SentimentLSTM class is defined as during training)
# model = SentimentLSTM(...)
model.load_state_dict(torch.load("/content/drive/MyDrive/sentiment_analysis/checkpoints/lstm_checkpoint_2.pt")["model_state_dict"])
model.to(device)
model.eval()

# Label mapping
idx_to_label = {0: "negative", 1: "neutral", 2: "positive"}  # Adjust based on your data

def predict(text):
    # Tokenize the text
    encoding = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    # Move to device
    input_ids = encoding["input_ids"].to(device)

    # Forward pass
    with torch.no_grad():
        outputs = model(input_ids)
        probs = nn.functional.softmax(outputs, dim=1)
        predicted_class = torch.argmax(probs, dim=1).item()

    label = idx_to_label[predicted_class]
    confidence = probs[0][predicted_class].item()

    return label, confidence

# 🔍 Example usage
text = "वो फ़िल्म बुरी थी"  # Replace with your sentence
label, confidence = predict(text)
print(f"Predicted: {label} (Confidence: {confidence:.2f})")


Predicted: negative (Confidence: 0.95)
